# Import 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from keras.src.trainers.data_adapters.data_adapter_utils import class_weight_to_sample_weights
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import panel as pn 
import plotly.express as px
import hvplot.pandas 

In [3]:
df = pd.read_csv("/Users/nguyentoan/Documents/Bank_Customer_Churn/Data/Bank_data.csv")

In [15]:
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,age_group,churn_str
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1,36-45,Exited (1)
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,36-45,Stayed (0)
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,36-45,Exited (1)
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0,36-45,Stayed (0)
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,36-45,Stayed (0)


In [16]:
df.shape

(10000, 14)

# Exploratory Data Analysis – EDA 

In [17]:
df.isnull().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
age_group           0
churn_str           0
dtype: int64

In [18]:
df.duplicated().sum()

np.int64(0)

In [4]:
import panel as pn 
import plotly.express as px

In [5]:
pn.extension('plotly')
colors = px.colors.sequential.Blues

### SECTION 1 

In [6]:
# ---- Churn Rate Overall ----
churn_rate = df["churn"].value_counts().reset_index()
churn_rate.columns = ["churn", "count"]

fig_churn_pie = px.pie(
    churn_rate,
    names="churn",
    values="count",
    title="Overall Churn Rate",
    color_discrete_sequence=px.colors.sequential.Teal
)

In [7]:
churn_country = df.groupby("country")["churn"].mean().reset_index()

fig_country = px.bar(
    churn_country,
    x="country",
    y="churn",
    title="Churn Rate by Country",
    text="churn",
    color_discrete_sequence=["#CFF4F6"]
)

fig_country.update_traces(
    texttemplate='%{text:.1%}',
    textposition='outside'
)

fig_country.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showgrid=False),
    yaxis=dict(
        showgrid=False,
        tickformat=".0%", 
        title="Churn Rate"
    ),
    showlegend=False
)

In [8]:
# Tính churn rate theo active_member
active_churn = pd.crosstab(
    df["active_member"],
    df["churn"],
    normalize="index"  
).reset_index()

# Melt data để plot plotly dễ hơn
active_churn = active_churn.melt(
    id_vars="active_member",
    var_name="churn",
    value_name="rate"
)

# Plot
fig_active = px.bar(
    active_churn,
    x="active_member",
    y="rate",
    color="churn",
    barmode="stack",
    title="Churn Rate by Active Member Status",
    color_discrete_sequence=px.colors.sequential.Teal
)

# Format text + layout
fig_active.update_traces(
    texttemplate="%{y:.1%}",
    textposition="inside"
)

fig_active.update_layout(
    plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        showgrid=False,
        tickvals=[0, 1],
        ticktext=["Inactive", "Active"]
    ),
    yaxis=dict(
        showgrid=False,
        tickformat=".0%",
        title="Churn Rate"
    ),
    legend_title="Churn Status"
)

In [9]:
# Tính churn rate theo số sản phẩm
product_churn = df.groupby("products_number")["churn"].mean().reset_index()

# Sort theo số sản phẩm
product_churn = product_churn.sort_values("products_number")

# Plot
fig_product = px.bar(
    product_churn,
    x="products_number",
    y="churn",
    title="Churn Rate by Number of Products",
    text="churn",
    color_discrete_sequence=["#CFF4F6"]
)

# Format label
fig_product.update_traces(
    texttemplate="%{text:.1%}",
    textposition="outside"
)

# Layout tuning
fig_product.update_layout(
    plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        title="Number of Products",
        showgrid=False,
        tickmode="array",   # ⭐ Quan trọng
        tickvals=[1,2,3,4],  # Chỉ hiển thị 1 → 4
        ticktext=["1","2","3","4"]
    ),
    yaxis=dict(
        title="Churn Rate",
        tickformat=".0%",
        showgrid=False
    ),
    showlegend=False
)

### SECTION 2 - Customer Behavior

In [10]:
# Phân nhóm tuổi theo đúng logic SQL
bins = [0, 24, 35, 45, 100]
labels = ['Under 25', '25-35', '36-45', '45+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)

# Tính toán tỉ lệ Churn theo từng nhóm (giống đoạn GROUP BY trong SQL)
age_group_analysis = df.groupby('age_group')['churn'].mean().reset_index()
age_group_analysis['churn_rate'] = age_group_analysis['churn'] * 100

fig_age = px.bar(
    age_group_analysis,
    x="age_group",
    y="churn_rate",
    title="Churn Rate by Age Group",
    text="churn_rate", # Hiển thị con số trên đầu cột
    color="churn_rate",
    color_continuous_scale=px.colors.sequential.Teal
)

# Tinh chỉnh giao diện Minimalist
fig_age.update_traces(
    texttemplate='%{text:.1f}%', 
    textposition='outside'
)

fig_age.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(title="Age Group", showgrid=False),
    yaxis=dict(
        title="Churn Rate (%)", 
        showgrid=False, 
        range=[0, max(age_group_analysis['churn_rate']) + 10] # Tạo không gian cho nhãn
    ),
    showlegend=False
)

In [11]:
df['churn_str'] = df['churn'].astype(str).map({'0': 'Stayed (0)', '1': 'Exited (1)'})

fig_credit = px.box(
    df,
    x="churn_str",
    y="credit_score",
    color="churn_str",
    title="Credit Score Distribution vs Churn Status",
    points="outliers", 
    notched=True,     
    color_discrete_map={
        'Stayed (0)': '#A8E6CF',   
        'Exited (1)': '#DCEDC1'    
    }
)

fig_credit.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(title="Customer Status", showgrid=False),
    yaxis=dict(title="Credit Score", showgrid=True, gridcolor='lightgrey'),
    showlegend=False
)

fig_credit.show()

In [12]:
fig_balance_salary = px.scatter(
    df,
    x="estimated_salary",
    y="balance",
    size="credit_score",   # thêm chiều dữ liệu
    color="churn",
    opacity=0.5,
    color_continuous_scale=px.colors.sequential.Blues,
    title="Balance vs Salary (Bubble = Credit Score)"
)

fig_balance_salary.update_layout(
    plot_bgcolor="white"
)

fig_balance_salary.show()

In [13]:
# ---- Tenure vs Churn ----
tenure_churn = df.groupby("tenure")["churn"].mean().reset_index()

fig_tenure = px.line(
    tenure_churn,
    x="tenure",
    y="churn",
    title="Tenure vs Churn Rate",
    markers=True,
    line_shape="spline",
    color_discrete_sequence=px.colors.sequential.Aggrnyl
)

# Create Dashboard

In [ ]:
title = pn.pane.Markdown(
    """
    # 🏦 Customer Churn Analytics Dashboard
    ## Business & Customer Insight Analysis
    """
)

section1 = pn.Column(
    "## 🟢 Business Performance",

    pn.GridBox(
        pn.pane.Plotly(fig_churn_pie),
        pn.pane.Plotly(fig_country),
        pn.pane.Plotly(fig_active),
        pn.pane.Plotly(fig_product),
        ncols=2,
        sizing_mode="stretch_width"
    )
)

# Section 2
section2 = pn.Column(
    "## 🔵 Customer Behavior Analysis",
    pn.Row(
        pn.pane.Plotly(fig_age),
        pn.pane.Plotly(fig_credit)
    ),
    pn.Row(
        pn.pane.Plotly(fig_balance_salary),
        pn.pane.Plotly(fig_tenure)
    )
)


# Final Dashboard
dashboard = pn.Tabs(
    ("Overview", section1),
    ("Customer Insight", section2)
)

pn.Column(title, dashboard).servable()
# panel serve Bank.ipynb

BokehModel(combine_events=True, render_bundle={'docs_json': {'da87406d-ae98-43d8-bfdc-7417f4b33cda': {'version…

# Code push data into MySql

In [ ]:
from sqlalchemy import create_engine

# MySQL connection info
username = "root"
password = "****"
host = "127.0.0.1"
port = "3306"
database = "Bank_DB"

# Create engine
engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}?charset=utf8mb4"
)
table_name = "bank_customers_churn"
df.to_sql(table_name, engine, if_exists="replace", index=False)

In [ ]:
df_json = df.copy()

In [ ]:
df_json.to_json(
    "bank_customers_churn.json",
    orient="records",
    indent=4
)